
# Partial - Discounted Cash Flow (DCF) Model - Machine Learning Preprocessing

After compiling various valuation metrics, and conducting exploratory analysis to generate insights into the SaaS industry, I came across something called the discounted cash flow model. Many of the valuation metrics surrounding debt, enterprise value, and revenue had strong correlations to other indications of company growth and value, so I found that a partial DCF model could help throughout my valuation research. 

A discounted cash flow model is a financial model that uses sensitivity analysis and free cash flows to predict a company's value in terms of TODAY'S money value. However, a DCF model relies on heavy assumptions. In particular, the most important assumption is the forecasting of unlevered free cash flows, or cash flows BEFORE accounting for interest payments, representing money available to all debt and equity holders. This value can paint a very clear picture of a company's intrinsic value. Afterwards, these forecasted values are "discounted" using a ratio that is calculated by an individual investor, with the output of a DCF model being an overall enterpise value. (If you remember, enterprise value is calculated using debt levels and market cap, but a DCF analysis can generate a more accurate enterprise, or present value.)

With all the data collected so far, my goal with this project is to make sure that investors have a tool that can help them create DCF models for company valuation. In particular, I want to take out the assumption in forecasting, by using machine learning models to predict unlevered free cash flows. Let's go over how I preprocessed the information needed to create a useful machine learning model to predict UFCFs.


## Step 1: Importing Libraries
This step imports the necessary Python libraries such as `numpy`, `pandas`, and machine learning libraries to build and evaluate the model.



## Step 2: Collect Financial Data
We collect the data required for the unlevered free cash flows using the basic formula: UFCF = EBIT × (1 − T) + Depreciation and amortization − Change in working capital − Capital expenditures. We want to use this information ALONG with our valuation metrics and other information to feed into our model. Once again, we use yfinance's API to get this data. We are also aiming to get historical information for each of these factors (annually) so that when we do create a machine learning model, it can easily make predictions based on historical trends. Ideally, the valuation metrics collected previously would be historical as well, but due to software and tool limitations, I could only get historical information for the UFCF factors, UFCF and non-ratio factors.


In [ ]:
import yfinance as yf
import pandas as pd
#straightforward functions, essentially using yfinance built in financial statements to get needed values!
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    return financials

def get_cash_flow_data(ticker):
    stock = yf.Ticker(ticker)
    cash_flow = stock.cashflow
    return cash_flow

def get_income_statement_data(ticker):
    stock = yf.Ticker(ticker)
    income_statement = stock.financials
    return income_statement

def get_balance_sheet_data(ticker):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    return balance_sheet
#get EBIT 
def get_ebit(ticker):
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials.T #transpose to get it in that tabular format with dates and values on right column
        if 'EBIT' not in financials.columns:
            raise ValueError("EBIT data is not available.")
        return financials['EBIT']
    except Exception as e:
        print(f"Error getting EBIT: {e}")
        return pd.Series() #holds sequence of EBIT values

def get_tax_rate(ticker): #tax rate is not used in output, but a helpful function for future use cases
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials.T #same transpose
        tax_rates = {}
        for date in financials.index:
            try:
                tax_expense = financials.loc[date, 'Tax Provision']
                pretax_income = financials.loc[date, 'Pretax Income']
                if pretax_income != 0:
                    tax_rate = tax_expense / pretax_income
                    tax_rates[date] = tax_rate
                else:
                    tax_rates[date] = None
            except KeyError as e:
                print(f"KeyError: {e} for date {date}")
                tax_rates[date] = None
            except ZeroDivisionError:
                tax_rates[date] = None
        tax_rates_df = pd.DataFrame(list(tax_rates.items()), columns=['Date', 'Tax Rate']) #dataframe or table of historical tax rates
        tax_rates_df.set_index('Date', inplace=True)
        return tax_rates_df['Tax Rate']
    except Exception as e:
        print(f"Error getting tax rate: {e}")
        return pd.Series()

def get_nopat(ticker):
    try:
        ebit_series = get_ebit(ticker)
        tax_rate_series = get_tax_rate(ticker)
        if ebit_series.empty or tax_rate_series.empty:
            raise ValueError("Unable to calculate NOPAT due to missing EBIT or tax rate data.")
        ebit_series = ebit_series.loc[tax_rate_series.index]
        tax_rate_series = tax_rate_series.loc[ebit_series.index]
        nopat_series = ebit_series * (1 - tax_rate_series) #match indexes (dates) and calculate based on matching index in the datafram table format
        return nopat_series
    except Exception as e:
        print(f"Error calculating NOPAT: {e}")
        return pd.Series()

def get_depreciation_amortization(ticker):
    try:
        stock = yf.Ticker(ticker)
        cashflow = stock.cashflow.T
        if 'Depreciation Amortization Depletion' not in cashflow.columns:
            raise ValueError("D&A data is not available.")
        return cashflow['Depreciation Amortization Depletion']
    except Exception as e:
        print(f"Error getting D&A: {e}")
        return pd.Series()

def get_capex(ticker):
    try:
        stock = yf.Ticker(ticker)
        cashflow = stock.cashflow.T
        if 'Capital Expenditure' not in cashflow.columns:
            raise ValueError("CapEx data is not available.")
        return cashflow['Capital Expenditure']
    except Exception as e:
        print(f"Error getting CapEx: {e}")
        return pd.Series()

def get_nwc(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet.T
        current_assets = balance_sheet.get('Current Assets', pd.Series())
        current_liabilities = balance_sheet.get('Current Liabilities', pd.Series())
        if not current_assets.empty and not current_liabilities.empty:
            nwc = current_assets - current_liabilities
        else:
            raise ValueError("Current Assets or Current Liabilities data is missing.")
        return nwc
    except Exception as e:
        print(f"Error getting NWC: {e}")
        return pd.Series()

def calculate_nwc_increase(ticker):
    try:
        nwc_series = get_nwc(ticker)
        if nwc_series.empty:
            raise ValueError("NWC data is not available.")
        nwc_change = nwc_series.diff()
        return nwc_change
    except Exception as e:
        print(f"Error calculating NWC increase: {e}")
        return pd.Series()

def calculate_ufcf(ticker):
    try:
        nopat_series = get_nopat(ticker)
        da_series = get_depreciation_amortization(ticker)
        capex_series = get_capex(ticker)
        nwc_increase_series = calculate_nwc_increase(ticker)
        if nopat_series.empty or da_series.empty or capex_series.empty or nwc_increase_series.empty:
            raise ValueError("One or more required data series are missing.")
        common_index = nopat_series.index.intersection(da_series.index).intersection(capex_series.index).intersection(nwc_increase_series.index) #merge series values based on common index which are the dates
        #align based on that common index which is date
        nopat_series = nopat_series.loc[common_index]
        da_series = da_series.loc[common_index]
        capex_series = capex_series.loc[common_index]
        nwc_increase_series = nwc_increase_series.loc[common_index]
        # UFCF = NOPAT + Depreciation & Amortization - Increase in NWC - Capital Expenditures
        ufcs = nopat_series + da_series - nwc_increase_series - capex_series #after merging based on index, you can actually calculate
        return ufcs
    except Exception as e:
        print(f"Error calculating UFCF: {e}")
        return pd.Series()

def process_tickers():
    results = []
    tickers = pd.read_csv('AllDatav2.csv')['Ticker']
    try:
        for ticker in tickers:
            try:
                ebitda = get_ebit(ticker)
                depamo = get_depreciation_amortization(ticker)
                capex = get_capex(ticker)
                increase_in_nwc = calculate_nwc_increase(ticker)
                ufcs = calculate_ufcf(ticker)
                result = {
                    'Ticker': ticker,
                    'Ebitda': ebitda,
                    'Depreciation/Amortization': depamo,
                    'Capital Expenditures': capex,
                    'Net Working Capital Increase Per Year': increase_in_nwc,
                    'UFCF': ufcs
                }
                results.append(result)
            except Exception as e:
                print(f"Failed to process {ticker}: {e}")
        results_df = pd.DataFrame(results)
        results_df.to_csv('DCFalgorithm.csv', index=False)
    except Exception as e:
        print(f"Error processing tickers: {e}")

process_tickers()


As you can see, this code has a variety of functions according to the unlevered free cash flow formula and uses the yfinance API to collect historical information. However, the output (attached below in a spreadsheet link) has only a limited amount of historical information. But anyways, lets use the Python pandas library to reformat our output spreadsheet into a machine learning-ready format, with columns Year, UFCF Factors, calculated UFCF and Ticker. 

In [ ]:
import pandas as pd

# Load the data
file_path = 'processed_data6.csv'  
data = pd.read_csv(file_path)

# Columns to reshape
metrics = [
    'Ebitda', 'Depreciation/Amortization', 'Capital Expenditures', 
    'Net Working Capital Increase Per Year', 'UFCF'
]

# Reshape data to long format
data_long = pd.wide_to_long(data, stubnames=metrics, i='Ticker', j='Year', sep='_', suffix='\d+').reset_index()

# Sorting by Ticker and Year
data_long = data_long.sort_values(by=['Ticker', 'Year'])

# Keeping only the third year's data for each Ticker
data_third_year = data_long.groupby('Ticker').nth(2).reset_index()

# Display the transformed data
print(data_third_year.head())

# Save the transformed data
output_path = 'processed_data_ml3.csv'  
data_third_year.to_csv(output_path, index=False)

import os
if os.path.exists(output_path):
    print(f'Transformed data has been saved to {output_path}')
else:
    print('Error in saving the transformed data')


At the end of this code, we calculate and store primary factors of unlevered free cash flows into a spreadsheet for further usage. Our UFCF values have also been stored. After reshaping and filtering the information, here is the spreadsheet output: https://docs.google.com/spreadsheets/d/1MeeeVvYWWZYqgge2MPyxj_VMEBmVgPVltq3V4NXZH_E/edit?usp=sharing A lot of the information for previous years is missing for each ticker, so for the rest of the project, I only used the most recent year UFCF formula components.

yfinance API is very limited when it comes to historical information. However, financial APIs are available on the internet, so I found an API that can output historical financial statement information all the way from 1980, if applicable for a company. I chose similar factors to our previous UFCF formula components, with the added benefit of extensive historical information for each component being available.

In [ ]:
import requests
import pandas as pd

# Define the API key and base URL for EODHD
API_KEY = ''
BASE_URL = 'https://eodhd.com/api/fundamentals/'

# Define a function to fetch financial data for a single ticker, with a specified start and end year
def fetch_financial_data(ticker, start_year=1980, end_year=2023): 
    url = f"{BASE_URL}{ticker}?api_token={API_KEY}&fmt=json"
    
    # Make the API request
    response = requests.get(url)
    
    # Convert the response to JSON
    data = response.json()
    
    # Extract relevant financial data
    financial_data = []
    if 'Financials' in data and 'Income_Statement' in data['Financials']: #all the financial statements!
        income_statements = data['Financials']['Income_Statement']['yearly']
        balance_sheets = data['Financials']['Balance_Sheet']['yearly']
        cash_flows = data['Financials']['Cash_Flow']['yearly']
        market_cap = data.get('Highlights', {}).get('MarketCapitalization', None)

        for year, details in income_statements.items():
            # Extract the year from the date string
            year_int = int(year[:4])
            if start_year <= year_int <= end_year:
                # Check if the year exists in balance sheets and cash flow statements
                if year in balance_sheets and year in cash_flows:
                    # Extract data from income statements, balance sheets, and cash flow statements
                    free_cash_flow = cash_flows[year].get('freeCashFlow', None)
                    ebitda = details.get('ebitda', None)
                    net_working_capital = balance_sheets[year].get('netWorkingCapital', None)

                    financial_data.append({ #our csv (spreadsheet) file contents!
                        'Year': year_int,
                        'Ticker': ticker,
                        'Total Revenue': details.get('totalRevenue', None), #revenue like our UFCF formula component
                        'Free Cash Flow': free_cash_flow, #similar to unlevered free cash flows but including interest and debt pyaments
                        'EBITDA': ebitda, #like our UFCF formula componenet
                        'Net Working Capital': net_working_capital, #like our UFCF formula component
                        'Market Capitalization': market_cap #an additional metric that could be beneficial in a historical format
                    })
    
    return financial_data #to be converted into a csv file

# Load the CSV file containing tickers
tickers_df = pd.read_csv('AllDatav2.csv')  

#our actual tickers
tickers = tickers_df['Ticker'].tolist()

# Fetch data for all tickers and compile into a DataFrame --> which will then be put into a spreadsheet
all_financial_data = []
for ticker in tickers:
    financial_data = fetch_financial_data(ticker)
    all_financial_data.extend(financial_data)

# Convert to DataFrame
df_financial = pd.DataFrame(all_financial_data)

# Sort the DataFrame by Ticker and Year
df_financial = df_financial.sort_values(by=['Ticker', 'Year'])

# Save the DataFrame to a CSV file
df_financial.to_csv('historical_financial_data_eodhd.csv', index=False)

# Display the DataFrame
print(df_financial)


The additional historical information can be found in the 'historical_financial_data_eodhd' tab on the spreadsheet!